In [117]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta
pd.set_option('display.max_rows',100)
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.width',1000)
pd.set_option('display.max_columns',20)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


## Cálculo do vol acumulado médio - periodo 60d, interval 5m

In [106]:
pd.set_option('display.max_rows',50)

def calc_volume(tick,period,interval):
    m5 = yf.Ticker(tick+".SA").history(period=period,interval=interval, prepost = True).reset_index()
    m5['ticker']=tick
    #trata horário de verão
    m5.loc[m5.Datetime.dt.date < pd.to_datetime("2021-02-22").date() , 'Datetime']  = (m5.Datetime - timedelta(minutes=60))
    m5.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    m5['Volume']=m5.Volume/100#1000000
    m5['vol_acumulado'] = m5.groupby(m5.Datetime.dt.date)['Volume'].cumsum()
    m5['hour']=m5.Datetime.dt.hour
    m5['minute']=m5.Datetime.dt.minute


    m5_media_qtd = pd.merge(
        m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
        ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
        ,how='inner', on=['hour','minute']
             )

    m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute'])
    m5_merge['pct_vol_acumulado'] = m5_merge.vol_acumulado / m5_merge.vol_acum_medio*100
    m5_merge=m5_merge.round(2)
    
    return m5_merge

In [114]:
ibr=pd.read_csv('./ibr.txt')#.tail(2)

period="60d"
interval="5m"

for i, row in ibr.iterrows(): 
    if i == 0:
        df = calc_volume(row.ticker,period,interval)
    else:
        df = df.append(calc_volume(row.ticker,period,interval))
df

,Datetime,Open,High,Low,Close,Volume,ticker,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum,pct_vol_acumulado
0,2021-02-09 10:05:00-02:00,41.55,41.87,41.20,41.20,0.0,MOSI3,0.0,10,5,388.44,9,0.00
1,2021-02-09 10:10:00-02:00,41.20,41.87,40.84,41.80,1999.0,MOSI3,1999.0,10,10,1274.11,9,156.89
2,2021-02-09 10:15:00-02:00,41.80,41.84,39.15,39.15,2981.0,MOSI3,4980.0,10,15,2129.11,9,233.90
3,2021-02-09 10:35:00-02:00,39.55,39.66,38.60,38.90,4755.0,MOSI3,9735.0,10,35,4211.56,9,231.15
4,2021-02-09 10:40:00-02:00,38.90,39.00,37.80,37.80,968.0,MOSI3,10703.0,10,40,4577.22,9,233.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4978,2021-02-24 17:30:00-03:00,31.18,31.26,31.15,31.24,1119.0,YDUQ3,21210.0,17,30,32146.67,3,65.98
4979,2021-02-24 17:35:00-03:00,31.24,31.24,31.15,31.15,531.0,YDUQ3,21741.0,17,35,32605.67,3,66.68
4980,2021-02-24 17:40:00-03:00,31.15,31.21,31.15,31.15,1018.0,YDUQ3,22759.0,17,40,33257.00,3,68.43
4981,2021-02-24 17:45:00-03:00,31.16,31.16,31.15,31.16,729.0,YDUQ3,23488.0,17,45,33791.00,3,69.51


In [118]:
df[df.pct_vol_acumulado > 700]

,Datetime,Open,High,Low,Close,Volume,ticker,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum,pct_vol_acumulado
4842,2021-02-24 10:00:00-03:00,9.92,10.10,9.92,10.08,354.0,AALR3,354.0,10,0,46.39,49,763.10
1005,2020-12-10 11:15:00-02:00,15.62,15.63,15.49,15.52,228275.0,ABEV3,281777.0,11,15,33283.24,59,846.60
1006,2020-12-10 11:20:00-02:00,15.52,15.55,15.42,15.51,24826.0,ABEV3,306603.0,11,20,35148.63,59,872.30
1007,2020-12-10 11:25:00-02:00,15.50,15.52,15.43,15.46,10532.0,ABEV3,317135.0,11,25,37222.73,59,851.99
1008,2020-12-10 11:30:00-02:00,15.50,15.52,15.47,15.51,24957.0,ABEV3,342092.0,11,30,41659.97,59,821.15
1009,2020-12-10 11:35:00-02:00,15.49,15.58,15.48,15.58,6500.0,ABEV3,348592.0,11,35,44855.32,59,777.15
1010,2020-12-10 11:40:00-02:00,15.57,15.62,15.55,15.60,6539.0,ABEV3,355131.0,11,40,47499.12,59,747.66
1011,2020-12-10 11:45:00-02:00,15.60,15.66,15.60,15.65,14226.0,ABEV3,369357.0,11,45,50536.75,59,730.87
1037,2020-12-10 13:55:00-02:00,15.62,15.63,15.55,15.59,504423.0,ABEV3,1007276.0,13,55,118604.29,59,849.27
1038,2020-12-10 14:00:00-02:00,15.58,15.65,15.58,15.60,10613.0,ABEV3,1017889.0,14,0,120593.30,60,844.07


## Merge do vol

In [212]:
m5_media_qtd = pd.merge(
    m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
    ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
    ,how='inner', on=['hour','minute']
         )
#m5_media_qtd = m5_media_qtd[m5_media_qtd.qtd_vol_acum > m5_media_qtd.qtd_vol_acum.max()*0.2]
m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute'])
m5_merge['pct_vol_acumulado'] = m5_merge.vol_acumulado / m5_merge.vol_acum_medio*100
m5_merge=m5_merge.round(2)
m5_merge.to_csv('./m5.csv')
m5_merge[(m5_merge.hour == 10) & (m5_merge.minute == 42) & (m5_merge.pct_vol_acumulado > 110)]

,Datetime,Open,High,Low,Close,Volume,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum,pct_vol_acumulado
224,2021-01-07 10:42:00-02:00,24.55,24.67,24.54,24.67,243.0,9659.0,10,42,4607.31,32,209.65
636,2021-01-11 10:42:00-02:00,24.40,24.43,24.40,24.40,50.0,35098.0,10,42,4607.31,32,761.79
1047,2021-01-13 10:42:00-02:00,25.79,25.94,25.76,25.94,569.0,7002.0,10,42,4607.31,32,151.98
3510,2021-02-01 10:42:00-02:00,24.70,24.80,24.70,24.80,119.0,5458.0,10,42,4607.31,32,118.46
3715,2021-02-02 10:42:00-02:00,26.40,26.49,26.40,26.40,246.0,7724.0,10,42,4607.31,32,167.65
3921,2021-02-03 10:42:00-02:00,27.79,27.83,27.78,27.83,116.0,6360.0,10,42,4607.31,32,138.04
6098,2021-02-22 10:42:00-03:00,28.59,28.85,28.59,28.85,376.0,6720.0,10,42,4607.31,32,145.86
6561,2021-02-24 10:42:00-03:00,31.85,31.89,31.61,31.64,347.0,7634.0,10,42,4607.31,32,165.69


## Diário

In [121]:
acao="BRFS3.SA"
df = yf.Ticker(acao).history(period="30d")
df['ultima_cotacao'] = yf.Ticker(acao).history(period="1m").tail(1).Close.iloc[0]
df['cotacao_dia_anterior'] = df['Close'].shift(1)
df['pct_do_dia'] = (df.Close - df.cotacao_dia_anterior) / df.cotacao_dia_anterior*100
df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
df.Volume = df.Volume/1000000
df['media_vol'] = df.Volume.mean()
df['var_vol'] = df.Volume / df.media_vol*100
df['pct_longo'] = (df.ultima_cotacao-df.Low)/df.Close*100
df=df.reset_index()
#df.sort_index(ascending=False).round(2)
df = df.sort_values(by='Date',ascending=True).round(2)
df[df.var_vol > 100]

,Date,Open,High,Low,Close,Volume,ultima_cotacao,cotacao_dia_anterior,pct_do_dia,media_vol,var_vol,pct_longo
0,2021-01-11,21.74,21.86,21.42,21.53,9.91,23.81,NaN,NaN,8.52,116.36,11.10
1,2021-01-12,21.65,22.29,21.31,21.36,13.41,23.81,21.53,-0.79,8.52,157.45,11.70
2,2021-01-13,21.40,21.56,20.93,21.24,12.08,23.81,21.36,-0.56,8.52,141.78,13.56
4,2021-01-15,21.00,21.27,20.62,20.76,9.55,23.81,21.10,-1.61,8.52,112.12,15.37
8,2021-01-21,20.42,20.56,19.86,20.05,8.97,23.81,20.41,-1.76,8.52,105.26,19.70
9,2021-01-22,19.85,20.72,19.63,20.69,9.57,23.81,20.05,3.19,8.52,112.28,20.20
10,2021-01-26,20.63,20.79,20.37,20.41,9.41,23.81,20.69,-1.35,8.52,110.46,16.85
11,2021-01-27,20.38,21.33,20.24,21.02,10.12,23.81,20.41,2.99,8.52,118.79,16.98
12,2021-01-28,20.92,21.78,20.77,21.65,9.80,23.81,21.02,3.00,8.52,115.07,14.04
13,2021-01-29,21.52,22.24,21.22,21.22,10.74,23.81,21.65,-1.99,8.52,126.05,12.21


## Gráfico

In [156]:
brf = yf.Ticker("BRFS3.SA").history(period='1y')
brf = brf.reset_index()

for i in ['Open','High','Close','Low']:
    brf[i] = brf[i].astype('float64')
    
fig = go.Figure(data=[go.Candlestick(x=brf['Date'],open=brf['Open'],high=brf['High'],low=brf['Low'],close=brf['Close'])])
#fig.show()